In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt,timedelta
from datetime import datetime,date
import matplotlib.pyplot as plt
import seaborn as sns
from kiblib.utils.db import DbConn
from kiblib.utils.evolution_stats import EvolutionActivite
from kiblib.utils import lucas_sns_params
import warnings
warnings.filterwarnings("ignore")
from kiblib.adherent import Adherent

# PROJET INFOS ADHERENTS + ANONYMISATION DE STATDB.STAT_IMPRESSION


Lors de la création de la table statdb.impression, la méthode était un peu bourrin et consistait à récupérer toutes les données sur les usagers dont le numéro de carte et l'identifiant Webkiosk. Or, pour rentrer en conformiter avec le RGPD, nous devons anonymiser ces données. Pour ne pas perdre des données relatives aux adhérents nous devons donc :
* extraire les données statistiques sur les adhérents (année, âge, fidélité etc...)
* anonymiser les données personnelles

In [2]:
db_conn = DbConn().create_engine()

query="""SELECT *
FROM statdb.stat_impressions si 
WHERE DATE(date_impression)>='2024-09-01'"""
impressions = pd.read_sql(query,db_conn)

In [3]:
impressions

,date_impression,wk_id,utilisateur,espace,poste,imprimante,nb_credits_utilises,nb_pages_imprimees,type_impression
0,2024-09-03 09:05:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B
1,2024-09-03 09:06:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B
2,2024-09-03 09:07:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B
3,2024-09-03 09:09:00,311,X0001945218,Rdc,rdc10-scan,RDC-NB,2,2,N&B
4,2024-09-03 09:10:00,14490,X0001945775,Rdc,rdc01,RDC-NB,1,1,N&B
...,...,...,...,...,...,...,...,...,...
10821,2024-10-26 17:48:00,618,X0002464091,Rdc,rdc14,RDC-COULEUR,4,1,COULEUR
10822,2024-10-26 17:48:00,96666,X0002428185,Rdc,rdc12,RDC-NB,1,1,N&B
10823,2024-10-26 17:51:00,100996,X0002470313,XIA,xia14-scan,PHARE-NB,1,1,N&B
10824,2024-10-26 17:51:00,100996,X0002470313,XIA,xia14-scan,PHARE-NB,1,1,N&B


In [4]:
impressions

,date_impression,wk_id,utilisateur,espace,poste,imprimante,nb_credits_utilises,nb_pages_imprimees,type_impression
0,2024-09-03 09:05:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B
1,2024-09-03 09:06:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B
2,2024-09-03 09:07:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B
3,2024-09-03 09:09:00,311,X0001945218,Rdc,rdc10-scan,RDC-NB,2,2,N&B
4,2024-09-03 09:10:00,14490,X0001945775,Rdc,rdc01,RDC-NB,1,1,N&B
...,...,...,...,...,...,...,...,...,...
10821,2024-10-26 17:48:00,618,X0002464091,Rdc,rdc14,RDC-COULEUR,4,1,COULEUR
10822,2024-10-26 17:48:00,96666,X0002428185,Rdc,rdc12,RDC-NB,1,1,N&B
10823,2024-10-26 17:51:00,100996,X0002470313,XIA,xia14-scan,PHARE-NB,1,1,N&B
10824,2024-10-26 17:51:00,100996,X0002470313,XIA,xia14-scan,PHARE-NB,1,1,N&B


In [5]:
query = """SELECT *
FROM koha_prod.borrowers"""

In [6]:
bloc_koha_info = pd.read_sql(query,db_conn)

In [7]:
bloc_koha_info

,borrowernumber,cardnumber,surname,firstname,title,othernames,initials,streetnumber,streettype,address,...,privacy_guarantor_checkouts,checkprevcheckout,updated_on,lastseen,lang,login_attempts,overdrive_auth_token,anonymized,autorenew_checkouts,primary_contact_method
0,25,X0002404233,SEYDI,Diompolon,None,,None,None,None,14 RUE BELL,...,0,inherit,2024-04-03 17:47:07,NaT,default,0,None,0,1,None
1,28,X0001659292,DUMONT,Nils,None,,None,None,None,21 RUE INGRES,...,0,inherit,2024-03-23 02:48:03,NaT,default,0,None,0,1,None
2,54,X0002159331,HAFIDI,Hassan,None,,None,None,None,8 RUE LEVERRIER,...,0,inherit,2024-01-27 17:25:43,NaT,default,0,None,0,1,None
3,61,X0002438696,BOUHAFS,Doria,None,,None,None,None,20 RUE DUGUESCLIN,...,0,inherit,2024-01-27 17:25:43,NaT,default,0,None,0,1,
4,70,X0002279381,BALAKODCH,Anas,None,,None,None,None,29 RUE ROLLIN,...,0,inherit,2024-01-27 17:25:43,NaT,default,0,None,0,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30494,82186,X0002809915,LEPOUTRE,Valentina,None,,None,None,None,29 Avenue Frederic Sauvage,...,0,inherit,2024-10-30 16:43:12,NaT,default,0,None,0,1,
30495,82187,X0002809922,BOURI,Ayoub,None,,None,None,None,1/17 rue Maxence Vandermeersch,...,0,inherit,2024-10-30 16:47:37,NaT,default,0,None,0,1,
30496,82188,X0002809939,BOURI,Manel,None,,None,None,None,1/17 rue Maxence Vandermeersch,...,0,inherit,2024-10-30 16:49:55,NaT,default,0,None,0,1,
30497,82189,X0002809946,BENTIFOUR,Nadjat,None,,None,None,None,39 Rue de Tourcoing,...,0,inherit,2024-10-30 17:13:03,NaT,default,0,None,0,1,


In [8]:

df = impressions.merge(bloc_koha_info,left_on='utilisateur',right_on='cardnumber',how='left')

In [10]:
adh = Adherent(db_conn=db_conn,df=df)

In [13]:
adh.get_adherent_statdb_data()

In [15]:
impressions_infos_adherents = adh.df

* age
* sexe*
* ville
* iris
* branchcode
* categorycode
* fidelite

In [23]:
list_col1 = impressions.columns.to_list()

In [25]:
list_col1

['date_impression',
 'wk_id',
 'utilisateur',
 'espace',
 'poste',
 'imprimante',
 'nb_credits_utilises',
 'nb_pages_imprimees',
 'type_impression']

In [28]:
list_col2= ['adh_inscription_carte_code',
            'adh_inscription_carte_personnalite_code',
            'adh_inscription_site_code',
            'adh_inscription_attribut_action_code',
            'adh_inscription_attribut_bus_code',
            'adh_inscription_attribut_collect_code',
            'adh_inscription_attribut_pcs_code',
            'adh_sexe_code',
            'adh_age',
            'adh_age_code',
            'adh_geo_ville',
            'adh_geo_ville_bm',
            'adh_geo_ville_limitrophe',
            'adh_geo_rbx_iris_code',
            'adh_inscription_nb_annees_adhesion']

In [29]:
cols2keep = list_col1 + list_col2

In [30]:
cols2keep

['date_impression',
 'wk_id',
 'utilisateur',
 'espace',
 'poste',
 'imprimante',
 'nb_credits_utilises',
 'nb_pages_imprimees',
 'type_impression',
 'adh_inscription_carte_code',
 'adh_inscription_carte_personnalite_code',
 'adh_inscription_site_code',
 'adh_inscription_attribut_action_code',
 'adh_inscription_attribut_bus_code',
 'adh_inscription_attribut_collect_code',
 'adh_inscription_attribut_pcs_code',
 'adh_sexe_code',
 'adh_age',
 'adh_age_code',
 'adh_geo_ville',
 'adh_geo_ville_bm',
 'adh_geo_ville_limitrophe',
 'adh_geo_rbx_iris_code',
 'adh_inscription_nb_annees_adhesion']

In [31]:
impressions_infos_adherents_ok = impressions_infos_adherents[cols2keep]

In [32]:
impressions_infos_adherents_ok['ad']

,date_impression,wk_id,utilisateur,espace,poste,imprimante,nb_credits_utilises,nb_pages_imprimees,type_impression,adh_inscription_carte_code,...,adh_inscription_attribut_collect_code,adh_inscription_attribut_pcs_code,adh_sexe_code,adh_age,adh_age_code,adh_geo_ville,adh_geo_ville_bm,adh_geo_ville_limitrophe,adh_geo_rbx_iris_code,adh_inscription_nb_annees_adhesion
0,2024-09-03 09:05:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B,BIBL,...,NaN,NaN,NC,a49,i10,CROIX,non,limitrophe,NaN,14.0
1,2024-09-03 09:06:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B,BIBL,...,NaN,NaN,NC,a49,i10,CROIX,non,limitrophe,NaN,14.0
2,2024-09-03 09:07:00,2551,X0002724195,Rdc,rdc07-scan,RDC-NB,1,1,N&B,BIBL,...,NaN,NaN,NC,a49,i10,CROIX,non,limitrophe,NaN,14.0
3,2024-09-03 09:09:00,311,X0001945218,Rdc,rdc10-scan,RDC-NB,2,2,N&B,BIBL,...,NaN,NaN,NC,a66,i12,ROUBAIX,ville_bm,limitrophe,595120401,19.0
4,2024-09-03 09:10:00,14490,X0001945775,Rdc,rdc01,RDC-NB,1,1,N&B,BIBL,...,NaN,NaN,NC,a53,i10,LILLE,ville_bm,non,NaN,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10821,2024-10-26 17:48:00,618,X0002464091,Rdc,rdc14,RDC-COULEUR,4,1,COULEUR,BIBL,...,NaN,NaN,NC,a53,i10,ROUBAIX,ville_bm,limitrophe,595120801,17.0
10822,2024-10-26 17:48:00,96666,X0002428185,Rdc,rdc12,RDC-NB,1,1,N&B,BIBL,...,NaN,NaN,NC,a16,i05,ROUBAIX,ville_bm,limitrophe,595120601,5.0
10823,2024-10-26 17:51:00,100996,X0002470313,XIA,xia14-scan,PHARE-NB,1,1,N&B,BIBL,...,NaN,NaN,NC,a54,i10,CROIX,non,limitrophe,NaN,4.0
10824,2024-10-26 17:51:00,100996,X0002470313,XIA,xia14-scan,PHARE-NB,1,1,N&B,BIBL,...,NaN,NaN,NC,a54,i10,CROIX,non,limitrophe,NaN,4.0


# PROJET ANALYSE RESAS

In [46]:
query = """SELECT borrowernumber,reservedate,waitingdate,expirationdate,cancellationdate
FROM stat_reserves sr 
WHERE YEAR(waitingdate) = '2024'
"""

In [47]:
resas = pd.read_sql(query,db_conn)

In [48]:
resas

,borrowernumber,reservedate,waitingdate,expirationdate,cancellationdate
0,13838,2022-03-02,2024-07-25,2024-07-25,None
1,8853,2022-11-15,2024-03-22,2024-03-22,None
2,4446,2022-12-01,2024-03-28,2024-03-28,None
3,8460,2022-12-02,2024-03-28,2024-03-28,2024-03-28
4,55924,2023-03-10,2024-04-23,2024-04-23,None
...,...,...,...,...,...
28296,37880,2024-10-30,2024-10-30,2024-10-30,None
28297,47649,2024-10-30,2024-10-30,2024-10-30,None
28298,79592,2024-10-30,2024-10-30,2024-10-30,None
28299,71832,2024-10-30,2024-10-30,2024-10-30,None


In [49]:
resas.loc[resas['expirationdate']==resas['cancellationdate'],['détail']] = 'oui'

In [54]:
resas[(resas['détail']!='oui')&
      (~resas['détail'].isna())
     ]

,borrowernumber,reservedate,waitingdate,expirationdate,cancellationdate,détail


# SPLIT BARCODE RECOLEMENT JEUNESSE

In [9]:
import pandas as pd
from datetime import datetime as dt
from datetime import date



ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'FROM koha_prod.items i \nWHERE i.ccode IS NOT NULL' at line 2")
[SQL: SELECT i.barcode, i.ccode,
FROM koha_prod.items i 
WHERE i.ccode IS NOT NULL]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [78]:
from kiblib.utils.db import DbConn

db_conn = DbConn().create_engine()

query = """SELECT i.barcode, i.ccode
FROM koha_prod.items i 
WHERE i.ccode IS NOT NULL"""

items = pd.read_sql(query,db_conn)

In [79]:
items

,barcode,ccode
0,C0001353993,JABEFAE
1,C0000653853,JABEFAE
2,C0003476991,JABEFAE
3,C0001499529,JABEFAE
4,C0001353935,JABEFAE
...,...,...
272949,C3100012416,PENRSZZ
272950,C3100012424,PENRSZZ
272951,C3100012426,PENRSZZ
272952,C3100012427,PENRSZZ


In [80]:
rec = pd.read_csv('/home/kibini/recolement/data/recolement/20230822_Jeunesse_Documentaires0-500_JDT80ZZ.txt',
                  sep=" ",
                  header=None)

rec.columns = ['barcode_recolement']

In [81]:
rec

,barcode_recolement
0,C2500021692
1,C2500021647
2,C2500030712
3,C2500019790
4,C2500004082
...,...
1896,C0003636135
1897,C0004556257
1898,C0005045932
1899,C0004090976


In [82]:
rec_info_ccodes = rec.merge(items,right_on='barcode',left_on='barcode_recolement',how='left')

In [83]:
rec_info_ccodes

,barcode_recolement,barcode,ccode
0,C2500021692,C2500021692,JDT00ZZ
1,C2500021647,C2500021647,JDT00ZZ
2,C2500030712,C2500030712,JDT00ZZ
3,C2500019790,C2500019790,JDT00ZZ
4,C2500004082,C2500004082,JDT00ZZ
...,...,...,...
1896,C0003636135,NaN,NaN
1897,C0004556257,C0004556257,JDT50ZZ
1898,C0005045932,C0005045932,JDT50ZZ
1899,C0004090976,NaN,NaN


In [84]:
values2keep = ['JDT00ZZ',
            'JDT10ZZ,',
            'JDT20ZZ',
            'JDT30ZZ',
            'JDT40ZZ',
            'JDT50ZZ'
           ]

In [90]:
barcode_info_ccodes.to_excel("/home/kibini/recolement/data/recolement/0_liste_docs_jeunesse.xlsx",index=False)